In [1]:
pip install kagglehub

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install transformers datasets sentence-transformers langchain chromadb

Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.3
    Uninstalling numpy-2.3.3:
      Successfully uninstalled numpy-2.3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.16.2 which is incompatible.
tensorflow 2.18.0 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.20.0 which is incompatible.
fuzzy-c-means 1.7.2 requires typer<0.10.0,>=0.9.0, but you have typer 0.19.2 which is incompatible.
en-core-sci-md 0.5.0 requires spacy<3.3.0,>=3.2.3, but you have spacy 3.8.4 which is incompatible.
N

In [3]:
pip install sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade "numpy>=2.0" "scipy>=1.14" scikit-learn transformers sentence-transformers chromadb


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached numpy-2.3.3-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.61.0 requires numpy<2.2,>=1.24, but you have numpy 2.3.3 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.3.3 which is incompatible.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.16.2 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.3.3 which is incompatible.
tensorflow 2.18.0 requires te

In [5]:
pip freeze > requirements.txt


Note: you may need to restart the kernel to use updated packages.


In [32]:
import kagglehub
import json
import os
import numpy as np
import pandas as pd
import json
import re
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm 
import ast 


In [33]:
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

In [34]:
# Download QASPER dataset using KaggleHub
dataset_path = kagglehub.dataset_download("thedevastator/qasper-nlp-questions-and-evidence")
print(f"Path to QASPER dataset: {dataset_path}")


Path to QASPER dataset: /home/dswdfb/.cache/kagglehub/datasets/thedevastator/qasper-nlp-questions-and-evidence/versions/2


In [35]:
#Importing QASPER dataset


test = pd.read_csv(f"{dataset_path}/test.csv")
train = pd.read_csv(f"{dataset_path}/train.csv")
validation = pd.read_csv(f"{dataset_path}/validation.csv")



We had issues with the Keggle CSV file for QASPER DATASET. Tried parsing it manually as well as using the nested objects, but it still gave errors. This may be because of the object is converted to a string and the string contains Object types like 'Array()' which cause the json parse error.

We are using the same data QASPER sets but via HuggingFace, which provides straightfaward json object to parse

### Load the QASPER from huggingface

In [36]:
qasper_ds = load_dataset("allenai/qasper", split="train")

In [37]:
qasper_ds

Dataset({
    features: ['id', 'title', 'abstract', 'full_text', 'qas', 'figures_and_tables'],
    num_rows: 888
})

In [38]:
rows = []
for paper in qasper_ds:
    paper_id = paper["id"]
    title    = paper["title"]
    abstract = paper["abstract"]

    # full_text is a dict of columns
    sec_names = paper["full_text"]["section_name"]
    sec_paras = paper["full_text"]["paragraphs"]
    full_text = "\n\n".join(
        f"{sec}\n" + "\n".join(p) for sec, p in zip(sec_names, sec_paras)
    )

    qas = paper["qas"]
    n_questions = len(qas["question"])

    for i in range(n_questions):
        question_id   = qas["question_id"][i]
        question_text = qas["question"][i]
        nlp_bg        = qas["nlp_background"][i]
        topic_bg      = qas["topic_background"][i]
        paper_read    = qas["paper_read"][i]
        search_query  = qas["search_query"][i]
        question_writer = qas["question_writer"][i]

        # answers is ALSO a dict of parallel lists
        answers_block = qas["answers"][i]
        for ans, ann_id, worker_id in zip(
            answers_block["answer"],
            answers_block["annotation_id"],
            answers_block["worker_id"]
        ):
            rows.append({
                "paper_id"        : paper_id,
                "title"           : title,
                "abstract"        : abstract,
                "full_text"       : full_text,
                "question_id"     : question_id,
                "question"        : question_text,
                "nlp_background"  : nlp_bg,
                "topic_background": topic_bg,
                "paper_read"      : paper_read,
                "search_query"    : search_query,
                "question_writer" : question_writer,
                "annotation_id"   : ann_id,
                "worker_id"       : worker_id,
                "unanswerable"    : ans["unanswerable"],
                "yes_no"          : ans["yes_no"],
                "free_form_answer": ans["free_form_answer"],
                "extractive_spans": "; ".join(ans["extractive_spans"]),
                "evidence"        : "; ".join(ans["evidence"]),
                "highlighted_evidence": "; ".join(ans["highlighted_evidence"])
            })

qasper_df = pd.DataFrame(rows)

In [39]:
qasper_df.head()

,paper_id,title,abstract,full_text,question_id,question,nlp_background,topic_background,paper_read,search_query,question_writer,annotation_id,worker_id,unanswerable,yes_no,free_form_answer,extractive_spans,evidence,highlighted_evidence
0,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,753990d0b621d390ed58f20c4d9e4f065f0dc672,What is the seed lexicon?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,31e85022a847f37c15fd0415f3c450c74c8e4755,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,False,None,a vocabulary of positive and negative predicat...,,The seed lexicon consists of positive and nega...,The seed lexicon consists of positive and nega...
1,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,753990d0b621d390ed58f20c4d9e4f065f0dc672,What is the seed lexicon?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,95da0a6e1b08db74a405c6a71067c9b272a50ff5,2cfd959e433f290bb50b55722370f0d22fe090b7,False,None,,seed lexicon consists of positive and negative...,The seed lexicon consists of positive and nega...,The seed lexicon consists of positive and nega...
2,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,9d578ddccc27dd849244d632dd0f6bf27348ad81,What are the results?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,1e5e867244ea656c4b7632628086209cf9bae5fa,2cfd959e433f290bb50b55722370f0d22fe090b7,False,None,Using all data to train: AL -- BiGRU achieved ...,,FLOAT SELECTED: Table 3: Performance of variou...,FLOAT SELECTED: Table 3: Performance of variou...
3,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,02e4bf719b1a504e385c35c6186742e720bcb281,How are relations used to propagate polarity?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,49a78a07d2eed545556a835ccf2eb40e5eee9801,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,False,None,"based on the relation between events, the sugg...",,"In this paper, we propose a simple and effecti...","As illustrated in Figure FIGREF1, our key idea..."
4,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,02e4bf719b1a504e385c35c6186742e720bcb281,How are relations used to propagate polarity?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,acd6d15bd67f4b1496ee8af1c93c33e7d59c89e1,2cfd959e433f290bb50b55722370f0d22fe090b7,False,None,cause relation: both events in the relation sh...,,"In this paper, we propose a simple and effecti...","As illustrated in Figure FIGREF1, our key idea..."


In [40]:
len(qasper_df)

2675

In [41]:
#Drop question rows where the answer is empty
qasper_df = qasper_df[qasper_df['free_form_answer'] != ""]

In [42]:
len(qasper_df)

622

In [43]:
qasper_df.head()

,paper_id,title,abstract,full_text,question_id,question,nlp_background,topic_background,paper_read,search_query,question_writer,annotation_id,worker_id,unanswerable,yes_no,free_form_answer,extractive_spans,evidence,highlighted_evidence
0,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,753990d0b621d390ed58f20c4d9e4f065f0dc672,What is the seed lexicon?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,31e85022a847f37c15fd0415f3c450c74c8e4755,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,False,None,a vocabulary of positive and negative predicat...,,The seed lexicon consists of positive and nega...,The seed lexicon consists of positive and nega...
2,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,9d578ddccc27dd849244d632dd0f6bf27348ad81,What are the results?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,1e5e867244ea656c4b7632628086209cf9bae5fa,2cfd959e433f290bb50b55722370f0d22fe090b7,False,None,Using all data to train: AL -- BiGRU achieved ...,,FLOAT SELECTED: Table 3: Performance of variou...,FLOAT SELECTED: Table 3: Performance of variou...
3,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,02e4bf719b1a504e385c35c6186742e720bcb281,How are relations used to propagate polarity?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,49a78a07d2eed545556a835ccf2eb40e5eee9801,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,False,None,"based on the relation between events, the sugg...",,"In this paper, we propose a simple and effecti...","As illustrated in Figure FIGREF1, our key idea..."
4,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,02e4bf719b1a504e385c35c6186742e720bcb281,How are relations used to propagate polarity?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,acd6d15bd67f4b1496ee8af1c93c33e7d59c89e1,2cfd959e433f290bb50b55722370f0d22fe090b7,False,None,cause relation: both events in the relation sh...,,"In this paper, we propose a simple and effecti...","As illustrated in Figure FIGREF1, our key idea..."
5,1909.00694,Minimally Supervised Learning of Affective Eve...,Recognizing affective events that trigger posi...,Introduction\nAffective events BIBREF0 are eve...,44c4bd6decc86f1091b5fc0728873d9324cdde4e,How big is the Japanese data?,two,unfamiliar,no,,c1fbdd7a261021041f75fbe00a55b4c386ebbbb4,36926a4c9e14352c91111150aa4c6edcc5c0770f,2cfd959e433f290bb50b55722370f0d22fe090b7,False,None,7000000 pairs of events were extracted from th...,,"As a raw corpus, we used a Japanese web corpus...","As a raw corpus, we used a Japanese web corpus..."


In [44]:
# keep only rows with a question
questions = (
    qasper_df.groupby("question_id")
        .first()
        .reset_index()[["question_id", "question", "paper_id", "free_form_answer", "evidence"]]
        .rename(columns={
            "question_id": "id",
            "free_form_answer": "answer",
            "evidence": "context"

        })
)

questions.head()

,id,question,paper_id,answer,context
0,0038b073b7cca847033177024f9719c971692042,How is the input triple translated to a slot-f...,1706.04115,"The relation R(x,y) is mapped onto a question ...",We show that it is possible to reduce relation...
1,00bcdffff7e055f99aaf1b05cf41c98e2748e948,What is the baseline method for the task?,1909.02764,For the emotion recognition from text they use...,"For the emotion recognition from text, we manu..."
2,00ef9cc1d1d60f875969094bb246be529373cb1d,What methodology is used to compensate for lim...,1904.07342,Influential tweeters ( who they define as indi...,The first data batch consists of tweets releva...
3,01123a39574bdc4684aafa59c52d956b532d2e53,By how much does their method outperform state...,1905.10247,"AE-HCN outperforms by 17%, AE-HCN-CNN outperfo...",The goal of this paper is to propose a novel O...
4,01dc6893fc2f49b732449dfe1907505e747440b0,What debate topics are included in the dataset?,1906.03538,"Ethics, Gender, Human rights, Sports, Freedom ...",FLOAT SELECTED: Figure 3: Distribution of clai...


In [45]:
questions.to_csv('processed_qasper_data.csv', index=False)

### Loading the HotPotQA Dataset from Kegglehub

In [46]:
# Download HotpotQA dataset using KaggleHub
path = kagglehub.dataset_download("jeromeblanchet/hotpotqa-question-answering-dataset")
print(f"Path to HotpotQA dataset: {path}")


Path to HotpotQA dataset: /home/dswdfb/.cache/kagglehub/datasets/jeromeblanchet/hotpotqa-question-answering-dataset/versions/1


In [47]:
#Importing HotPotQA Dataset
import json

with open(f"{path}/hotpot_dev_distractor_v1.json", "r") as f:
    hotpot_data = json.load(f)

print(len(hotpot_data))        
print(hotpot_data[0].keys())   

7405
dict_keys(['_id', 'answer', 'question', 'supporting_facts', 'context', 'type', 'level'])


In [48]:
import pandas as pd

rows = []
for ex in hotpot_data:
    rows.append({
        "id": ex["_id"],
        "question": ex["question"],
        "answer": ex["answer"],
        "context": ex["context"],
        "supporting_facts": ex["supporting_facts"]
    })

hotpot_df = pd.DataFrame(rows)

In [49]:
hotpot_df.head()

,id,question,answer,context,supporting_facts
0,5a8b57f25542995d1e6f1371,Were Scott Derrickson and Ed Wood of the same ...,yes,"[[Ed Wood (film), [Ed Wood is a 1994 American ...","[[Scott Derrickson, 0], [Ed Wood, 0]]"
1,5a8c7595554299585d9e36b6,What government position was held by the woman...,Chief of Protocol,"[[Meet Corliss Archer, [Meet Corliss Archer, a...","[[Kiss and Tell (1945 film), 0], [Shirley Temp..."
2,5a85ea095542994775f606a8,"What science fantasy young adult series, told ...",Animorphs,"[[Andre Norton Award, [The Andre Norton Award ...","[[The Hork-Bajir Chronicles, 0], [The Hork-Baj..."
3,5adbf0a255429947ff17385a,Are the Laleli Mosque and Esma Sultan Mansion ...,no,"[[Esma Sultan (daughter of Abdülaziz), [Esma S...","[[Laleli Mosque, 0], [Esma Sultan Mansion, 0]]"
4,5a8e3ea95542995a26add48d,"The director of the romantic comedy ""Big Stone...","Greenwich Village, New York City","[[Just Another Romantic Wrestling Comedy, [Jus...","[[Big Stone Gap (film), 0], [Adriana Trigiani,..."


In [52]:
hotpot_df.to_csv('processed_hotpot_df.csv', index=False)

## Step 1: Summarization

### We will store context in ChromaDb with chunking, for this task we will use 256 as our chunk size

In [53]:
def ingest_qasper_to_chroma(qasper_df, chroma_collection, character_splitter, token_splitter):
    for id, group in qasper_df.groupby("id"):
        evidence = str(group.iloc[0]["context"])
    
        char_chunks = character_splitter.split_text(evidence)
    
        token_chunks = []
        for chunk in char_chunks:
            token_chunks.extend(token_splitter.split_text(chunk))
    
        if not token_chunks:
            print(f"Skipping paper {paper_id}: no chunks produced.")
            continue
        
        ids = [f"{id}_{i}" for i in range(len(token_chunks))]
        question_ids = group["question_id"].tolist()
        metadatas = [
            {
                "question_id": id,
            }
            for _ in token_chunks
        ]
    
        chroma_collection.add(
            documents=token_chunks,
            ids=ids,
            metadatas=metadatas
        )
    
    print("All papers processed and stored in Chroma.")


In [65]:
def ingest_hotpot_to_chroma(hotpot_df, chroma_collection, character_splitter, token_splitter):

    for q_id, group in tqdm(hotpot_df.groupby("id"), desc="Processing Hotpot QA"):

        row_context = group.iloc[0]["context"]

        if isinstance(row_context, str):
            try:
                row_context = ast.literal_eval(row_context)
            except Exception:
                row_context = [("No Title", [row_context])]

        context_blocks = []
        for title, paragraphs in row_context:
            section_text = "\n".join(paragraphs) if isinstance(paragraphs, list) else str(paragraphs)
            context_blocks.append(f"{title}\n{section_text}")

        full_text = "\n\n".join(context_blocks)
        char_chunks = character_splitter.split_text(full_text)

        token_chunks = []
        for chunk in char_chunks:
            token_chunks.extend(token_splitter.split_text(chunk))

        if not token_chunks:
            print(f"Skipping question {q_id}: no context found produced.")
            continue

        ids = [f"{q_id}_{i}" for i in range(len(token_chunks))]
        metadatas = [{"question_id": q_id} for _ in token_chunks]

        chroma_collection.add(
            documents=token_chunks,
            ids=ids,
            metadatas=metadatas
        )

    print("All HotpotQA questions processed and stored in Chroma.")


In [66]:
DATASET_HOTPOT = "HotpotQA"
DATASET_QASPER = "Qasper"

TOKEN_CHUNK_SIZE = 256
TOKEN_CHUNK_OVERLAP = 10

CHAR_CHUNK_SIZE = 1000
CHAR_CHUNK_OVERLAP = 10

In [67]:
def run_data_loader_pipeline(DATASET_TYPE, collection, dataframe):
    
    character_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ". ", " ", ""],
        chunk_size=CHAR_CHUNK_SIZE,
        chunk_overlap=CHAR_CHUNK_OVERLAP
    )

    token_splitter = SentenceTransformersTokenTextSplitter(
        chunk_overlap=TOKEN_CHUNK_OVERLAP,
        tokens_per_chunk=TOKEN_CHUNK_SIZE
    )

    if DATASET_TYPE == DATASET_QASPER:
        ingest_qasper_to_chroma(dataframe, collection, character_splitter, token_splitter)
    elif DATASET_TYPE == DATASET_HOTPOT:
        ingest_hotpot_to_chroma(dataframe, collection, character_splitter, token_splitter)
    else:
        print("Invalid Dataset Type")    

In [68]:
def create_chroma_collections(db_path: str, configs: dict, embedding_models: list):
    
    client = chromadb.PersistentClient(path=db_path)
    collections = {}

    for dataset_name, cfg in configs.items():
        base_name = cfg["collection"]
        collections[dataset_name] = {}

        for model_name in embedding_models:
            short_model = model_name.split("/")[-1]
            collection_name = f"{base_name}_{short_model}"

            try:
                client.delete_collection(name=collection_name)
                print(f"Deleted old collection: {collection_name}")
            except Exception:
                pass

            # create embedding function for this model
            embedding_fn = SentenceTransformerEmbeddingFunction(model_name=model_name)

            # create and store the collection
            coll = client.create_collection(
                name=collection_name,
                embedding_function=embedding_fn
            )

            collections[dataset_name][short_model] = coll
            print(f" Created collection '{collection_name}' using '{model_name}'")

    return collections


In [ ]:
DATASET_CONFIGS = {
    "HotpotQA": {
        "collection": "hpqa_data_collection"
    },
    "Qasper": {
        "collection": "qasper_data_collection"
    }
}

EMBEDDING_MODELS = [
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-mpnet-base-v2",
    "sentence-transformers/all-MiniLM-L12-v2"
]

collections = create_chroma_collections("./ChromaDb", DATASET_CONFIGS, EMBEDDING_MODELS)

# Example keys: collections["HotpotQA"]["all-MiniLM-L6-v2"]

hotpot_df = pd.read_csv("processed_hotpot_df.csv")
qasper_df = pd.read_csv("processed_qasper_data.csv")

for model_key, hotpot_collection in collections["HotpotQA"].items():
    print(f"Loading HotpotQA with model: {model_key}")
    run_data_loader_pipeline(DATASET_HOTPOT, hotpot_collection, hotpot_df)

for model_key, qasper_collection in collections["Qasper"].items():
    print(f"Loading Qasper with model: {model_key}")
    run_data_loader_pipeline(DATASET_QASPER, qasper_collection, qasper_df)


Deleted old collection: hpqa_data_collection_all-MiniLM-L6-v2
 Created collection 'hpqa_data_collection_all-MiniLM-L6-v2' using 'sentence-transformers/all-MiniLM-L6-v2'
Deleted old collection: hpqa_data_collection_all-mpnet-base-v2
 Created collection 'hpqa_data_collection_all-mpnet-base-v2' using 'sentence-transformers/all-mpnet-base-v2'
Deleted old collection: hpqa_data_collection_all-MiniLM-L12-v2
 Created collection 'hpqa_data_collection_all-MiniLM-L12-v2' using 'sentence-transformers/all-MiniLM-L12-v2'
Deleted old collection: qasper_data_collection_all-MiniLM-L6-v2
 Created collection 'qasper_data_collection_all-MiniLM-L6-v2' using 'sentence-transformers/all-MiniLM-L6-v2'
Deleted old collection: qasper_data_collection_all-mpnet-base-v2
 Created collection 'qasper_data_collection_all-mpnet-base-v2' using 'sentence-transformers/all-mpnet-base-v2'
Deleted old collection: qasper_data_collection_all-MiniLM-L12-v2
 Created collection 'qasper_data_collection_all-MiniLM-L12-v2' using 'sen

Processing Hotpot QA: 100%|██████████| 7405/7405 [13:32<00:00,  9.12it/s]  


All HotpotQA questions processed and stored in Chroma.
Loading HotpotQA with model: all-mpnet-base-v2


Processing Hotpot QA:  10%|█         | 754/7405 [05:55<51:37,  2.15it/s]  

In [ ]:
a